In [1]:
!which jupyter
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm import tqdm
import re

/home/ubuntu/thesis_env2/bin/jupyter


In [2]:
# NER File
df_ner = pd.read_pickle('./data/df_ner_matched_210913.pickle')
print(len(df_ner))

369045


In [3]:
df_ner.head(1)

,author,date,domain,title,url,content,topic_area,org_names,filtered_names,filtered_names_match
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,"[BA, Boeing, CAT, CMI, Caterpillar, Cummins In...","[Boeing, Caterpillar, Cummins Inc, Deere & Com...","[Caterpillar Inc., Cummins Inc., Deere & Compa..."


In [4]:
# Sentiment Analysis
df_sent = pd.read_pickle('./data/df_sentiment.pickle')
df_sent['finbert'] = df_sent['finbert'].str.upper()

sentiment_match = np.array(df_sent[['finbert', 'distilbert', 'roberta']])
sentiment_match = np.all(np.equal(sentiment_match, sentiment_match[:,:1]), axis=1)
df_sent['sentiment_match'] = sentiment_match
print(len(df_sent))

# Topics
df_topics = pd.read_pickle("./data/df_with_topics")
df_topics['topic_label_str'] = ['_'.join(map(str, l)) for l in df_topics['topic_label']]
print(len(df_topics))

365200
365200


In [5]:
# List of all companies in filterned_names_match
companies = [comp for sublist in df_ner['filtered_names_match'].tolist() for comp in sublist]
#companies = sorted(set(companies))

In [6]:
list(filter(lambda x: "Wesfarmers" in x, companies))

['Wesfarmers Limited',
 'Wesfarmers Limited',
 'Wesfarmers Limited',
 'Wesfarmers Limited',
 'Wesfarmers Limited',
 'Wesfarmers Limited',
 'Wesfarmers Limited',
 'Wesfarmers Limited']

In [7]:
from collections import Counter
companies_count = Counter(companies)
companies_count = pd.DataFrame.from_dict(companies_count, orient='index', columns=['count']).reset_index()
companies_count.columns = ['company', 'count']
companies_count.head(1)

,company,count
0,Caterpillar Inc.,526


In [8]:
companies_count[companies_count['company'].str.lower().str.contains('samsung')]

,company,count


In [9]:
companies_count.sort_values(by='count', ascending=False)[100:150]

,company,count
864,First Property Group plc,569
371,NVIDIA Corporation,560
90,"Marriott International, Inc.",552
155,The Procter & Gamble Company,544
0,Caterpillar Inc.,526
1292,"Hertz Global Holdings, Inc.",519
182,BHP Group,517
311,IG Group Holdings plc,516
483,"Liverpool Group, Inc.",511
801,"American Software, Inc.",510


In [10]:
df_combined = pd.merge(df_sent, df_topics[['topic_id', 'topic_label_str', 'topic_score']], how='left', left_index=True, right_index=True)
df_combined = pd.merge(df_combined, df_ner[['filtered_names_match']], how='left', left_index=True, right_index=True)
df_combined['nb_companies'] = df_combined['filtered_names_match'].apply(lambda x: len(x))

In [11]:
df_combined

,author,date,domain,title,url,content,topic_area,content_processed,distilbert,finbert,roberta,sentiment_match,topic_id,topic_label_str,topic_score,filtered_names_match,nb_companies
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,end year corner past time think positioning fo...,POSITIVE,NEUTRAL,POSITIVE,False,970,,0.437342,"[Caterpillar Inc., Cummins Inc., Deere & Compa...",5
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business,labor markets closely watched segment economy ...,NEGATIVE,NEGATIVE,POSITIVE,False,723,,0.383544,[Cintas Corporation],1
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business,forgiven think tesla nasdaq tsla little big pi...,POSITIVE,POSITIVE,POSITIVE,True,436,,0.496218,"[Tesla, Inc.]",1
3,Roberto Torres,2020-01-03,ciodive,"On the road to AI adoption, execs grapple with...",https://www.ciodive.com/news/ai-adoption-execs...,CIOs kicked off 2019 with AI as an item to wat...,tech,cios kicked ai item watch competition agenda a...,POSITIVE,NEUTRAL,POSITIVE,False,234,,0.750307,"[Gartner, Inc.]",1
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer,coronavirus pandemic life returns normal celeb...,POSITIVE,NEUTRAL,POSITIVE,False,28,luxury_fashion_retailers_covid_stimulus,0.450103,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369042,Polly Harrison,2020-12-31,thefintechtimes,A Human Touch Will Be a Competitive Edge After...,https://thefintechtimes.com/53867-2/,Niels Pedersen is a Chartered Accountant and S...,finance,niels pedersen chartered accountant senior lec...,POSITIVE,NEUTRAL,POSITIVE,False,67,fintech,0.588367,[Lloyds Banking Group plc],1
369043,NaN,2020-12-31,marketscreener,Datametrex AI : Announces Deploying NexaSecuri...,https://www.marketscreener.com/quote/stock/DAT...,Datametrex AI Limited (TSXV: DM) (FSE: D4G) (O...,business,datametrex ai limited tsxv dm fse d4g otc pink...,POSITIVE,NEUTRAL,POSITIVE,False,696,,0.519550,"[American International Holdings Corp., Datame...",2
369044,Polly Harrison,2020-12-31,thefintechtimes,"US Payments: Smart Pension, Episode Six, PAAY ...",https://thefintechtimes.com/us-payments-smart-...,"This December, The Fintech Times is asking ind...",finance,december fintech times asking industry leaders...,POSITIVE,NEUTRAL,POSITIVE,False,67,fintech,0.617454,[],0
369045,NaN,2020-12-31,marketscreener,"WESTWATER RESOURCES, INC. : Entry into a Mater...",https://www.marketscreener.com/quote/stock/WES...,Item 1.01Entry into a Material Definitive Agre...,business,item 101entry material definitive agreement de...,POSITIVE,NEUTRAL,POSITIVE,False,421,,0.580690,"[American International Holdings Corp., Westwa...",2


In [12]:
df_combined[df_combined['topic_label_str'] != ""]

,author,date,domain,title,url,content,topic_area,content_processed,distilbert,finbert,roberta,sentiment_match,topic_id,topic_label_str,topic_score,filtered_names_match,nb_companies
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer,coronavirus pandemic life returns normal celeb...,POSITIVE,NEUTRAL,POSITIVE,False,28,luxury_fashion_retailers_covid_stimulus,0.450103,[],0
18,By Sui-Lee Wee and Donald G. McNeil Jr.,2020-01-09,nytimes,China Identifies New Virus Causing Pneumoniali...,https://www.nytimes.com/2020/01/08/health/chin...,HONG KONG — Chinese researchers say they have ...,business,hong kong chinese researchers identified new v...,NEGATIVE,NEUTRAL,NEGATIVE,False,1,nytimes_updates,0.553961,[],0
24,By Amy Qin and Javier C. Hernández,2020-01-11,nytimes,China Reports First Death From New Virus,https://www.nytimes.com/2020/01/10/world/asia/...,HONG KONG — Chinese state media on Saturday re...,business,hong kong chinese state media saturday reporte...,NEGATIVE,NEGATIVE,NEGATIVE,True,1,nytimes_updates,0.523607,[],0
27,Thomas Hughes,2020-01-13,marketbeat,"Lululemon Growth Strategy Gains Traction, Guid...",https://www.marketbeat.com/originals/lululemon...,Lululemon (LULU) just raised its 4th quarter g...,business,lululemon lulu raised 4th quarter guidance sha...,POSITIVE,POSITIVE,POSITIVE,True,89,clothing_retailers_store_closures,0.551399,"[NIKE, Inc.]",1
29,Chris Markoch,2020-01-13,marketbeat,How to Approach Bank Stocks as Earnings Season...,https://www.marketbeat.com/originals/how-to-ap...,Some of the nation’s largest banks such as JP ...,business,nation largest banks jp morgan chase nysejpm b...,POSITIVE,NEUTRAL,POSITIVE,False,92,banks,0.449021,"[Bank of America Corporation, Capital One Fina...",11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369040,By Rachel Fakhry,2020-12-31,greenbiz,"If we want hydrogen to live up to its promise,...",https://www.greenbiz.com/article/if-we-want-hy...,This year saw a slew of global hydrogen announ...,environment,year saw slew global hydrogen announcements ea...,NEGATIVE,NEUTRAL,POSITIVE,False,30,climate_change_carbon_emissions,0.537654,"[Hydrogen Group Plc, WEC Energy Group, Inc.]",2
369041,Gina Clarke,2020-12-31,thefintechtimes,Behind the Idea: Kidbrooke,https://thefintechtimes.com/behind-the-idea-ki...,After nearly 20 years of an international care...,finance,nearly years international career insurance ca...,POSITIVE,NEUTRAL,POSITIVE,False,67,fintech,0.642278,[],0
369042,Polly Harrison,2020-12-31,thefintechtimes,A Human Touch Will Be a Competitive Edge After...,https://thefintechtimes.com/53867-2/,Niels Pedersen is a Chartered Accountant and S...,finance,niels pedersen chartered accountant senior lec...,POSITIVE,NEUTRAL,POSITIVE,False,67,fintech,0.588367,[Lloyds Banking Group plc],1
369044,Polly Harrison,2020-12-31,thefintechtimes,"US Payments: Smart Pension, Episode Six, PAAY ...",https://thefintechtimes.com/us-payments-smart-...,"This December, The Fintech Times is asking ind...",finance,december fintech times asking industry leaders...,POSITIVE,NEUTRAL,POSITIVE,False,67,fintech,0.617454,[],0


In [13]:
df_combined.loc[306]['filtered_names_match']

['Advanced Micro Devices, Inc.',
 'American Airlines Group Inc.',
 'Comcast Corporation',
 'Intel Corporation',
 'Southwest Airlines Co.']

In [14]:
selection = ['American Airlines Group Inc.']
df_selection = df_combined.iloc[pd.DataFrame(df_combined['filtered_names_match'].tolist()).isin(selection).any(1).values]
len(df_selection)
df_selection[df_selection['sentiment_match'] == True].groupby(['topic_id', 'topic_label_str', 'distilbert']).count()[['date']].sort_values(by='date', ascending=False)

,,,date
topic_id,topic_label_str,distilbert,
34,airlines,NEGATIVE,213
458,airlines_cost_cutting_reduced_flights,NEGATIVE,51
34,airlines,POSITIVE,45
573,airlines_flights_suspended,NEGATIVE,39
317,airlines_covid_19_safety_measures,NEGATIVE,38
...,...,...,...
138,,NEGATIVE,1
383,,POSITIVE,1
387,,NEGATIVE,1


In [15]:
selection = ['Carnival plc']
df_selection = df_combined.iloc[pd.DataFrame(df_combined['filtered_names_match'].tolist()).isin(selection).any(1).values]
len(df_selection)
df_selection[df_selection['sentiment_match'] == True].groupby(['topic_id', 'topic_label_str', 'distilbert']).count()[['date']].sort_values(by='date', ascending=False)

,,,date
topic_id,topic_label_str,distilbert,
103,cruises,NEGATIVE,121
10,general_stock_updates,NEGATIVE,35
103,cruises,POSITIVE,27
551,cruises_passengers_covid_19,NEGATIVE,16
732,general_stock_updates,NEGATIVE,13
...,...,...,...
357,,POSITIVE,1
321,,POSITIVE,1
319,,POSITIVE,1


In [16]:
selection = ['NVIDIA Corporation']
df_selection = df_combined.iloc[pd.DataFrame(df_combined['filtered_names_match'].tolist()).isin(selection).any(1).values]
len(df_selection)
df_selection[df_selection['sentiment_match'] == True].groupby(['topic_id', 'topic_label_str', 'distilbert']).count()[['date']].sort_values(by='date', ascending=False)

date
topic_id topic_label_str           distilbert      
242      amd_nvidia_microprocessor POSITIVE      22
5        us_shares                 POSITIVE       9
242      amd_nvidia_microprocessor NEGATIVE       9
460      us_shares                 NEGATIVE       8
1027                               NEGATIVE       8
...                                             ...
535                                NEGATIVE       1
550                                NEGATIVE       1
                                   POSITIVE       1
557                                NEGATIVE       1
1692     us_shares_down            NEGATIVE       1

[74 rows x 1 columns]

In [17]:
selection = ['Advanced Micro Devices, Inc.']
df_selection = df_combined.iloc[pd.DataFrame(df_combined['filtered_names_match'].tolist()).isin(selection).any(1).values]
len(df_selection)
df_selection[df_selection['sentiment_match'] == True].groupby(['topic_id', 'topic_label_str', 'distilbert']).count()[['date']].sort_values(by='date', ascending=False)

date
topic_id topic_label_str                               distilbert      
242      amd_nvidia_microprocessor                     POSITIVE      27
                                                       NEGATIVE      25
434      us_shares                                     POSITIVE      15
812      us_shares                                     NEGATIVE      13
5        us_shares                                     NEGATIVE      10
1660     company_specific_amd_nvidia_microprocessor    POSITIVE       9
460      us_shares                                     NEGATIVE       8
1355                                                   NEGATIVE       4
1308                                                   NEGATIVE       4
1099                                                   NEGATIVE       3
958                                                    NEGATIVE       3
698                                                    NEGATIVE       3
382                                                    NEGATIVE       3
315                                                    NEGATIVE       3
1692     us_shares_down                                NEGATIVE       3
94       result_announcements                          POSITIVE       3
112      general_stock_updates                         NEGATIVE       2
94       result_announcements                          NEGATIVE       2
1047                                                   POSITIVE       2
434      us_shares                                     NEGATIVE       2
425                                                    POSITIVE       2
5        us_shares                                     POSITIVE       2
698                                                    POSITIVE       2
1365                                                   NEGATIVE       1
40       general_stock_updates                         NEGATIVE       1
1315                                                   POSITIVE       1
1314     semiconductor_tech_company_china_supply_chain POSITIVE       1
66       etf                                           POSITIVE       1
1386                                                                  1
1477                                                   NEGATIVE       1
1197                                                   POSITIVE       1
1103                                                   POSITIVE       1
79       general_stock_updates                         NEGATIVE       1
1081                                                   NEGATIVE       1
1657     credit_cards_digital_payments                 POSITIVE       1
977                                                    POSITIVE       1
842                                                    POSITIVE       1
182                                                    POSITIVE       1
192                                                    POSITIVE       1
550                                                    POSITIVE       1
230                                                    NEGATIVE       1
240                                                    NEGATIVE       1
182                                                    NEGATIVE       1
120                                                    POSITIVE       1
262                                                    NEGATIVE       1
120                                                    NEGATIVE       1
315                                                    POSITIVE       1
118                                                    POSITIVE       1
382                                                    POSITIVE       1
118                                                    NEGATIVE       1
112      general_stock_updates                         POSITIVE       1
460      us_shares                                     POSITIVE       1
488                                                    POSITIVE       1
546                                                    NEGATIVE       1
550                                

In [18]:
selection = ["Macy's, Inc."]
df_selection = df_combined.iloc[pd.DataFrame(df_combined['filtered_names_match'].tolist()).isin(selection).any(1).values]
len(df_selection)
df_selection[df_selection['sentiment_match'] == True].groupby(['topic_id', 'topic_label_str', 'distilbert']).count()[['date']].sort_values(by='date', ascending=False)

,,,date
topic_id,topic_label_str,distilbert,
322,,NEGATIVE,26
77,retailers_bankruptcy,NEGATIVE,15
89,clothing_retailers_store_closures,NEGATIVE,10
1162,,NEGATIVE,9
16,us_unemployment,NEGATIVE,8
322,,POSITIVE,7
173,,NEGATIVE,5
1,nytimes_updates,NEGATIVE,4
89,clothing_retailers_store_closures,POSITIVE,4


In [19]:
selection = ['Uber Technologies, Inc.']
df_selection = df_combined.iloc[pd.DataFrame(df_combined['filtered_names_match'].tolist()).isin(selection).any(1).values]
len(df_selection)
df_selection[df_selection['sentiment_match'] == True].groupby(['topic_id', 'topic_label_str', 'distilbert']).count()[['date']].sort_values(by='date', ascending=False)

date
topic_id topic_label_str               distilbert      
514                                    NEGATIVE      34
979                                    NEGATIVE      13
1188                                   NEGATIVE      12
248                                    NEGATIVE       9
915                                    NEGATIVE       8
...                                                 ...
586                                    NEGATIVE       1
                                       POSITIVE       1
5        us_shares                     POSITIVE       1
703                                    NEGATIVE       1
1657     credit_cards_digital_payments POSITIVE       1

[73 rows x 1 columns]

In [20]:
df_combined['nb_comp_0'] = df_combined['nb_companies'].apply(lambda x: np.where(x==0, 1, 0))

In [21]:
topic_comp_check = df_combined.groupby('topic_id').agg({'topic_id': 'count', 'nb_comp_0': 'sum'})
topic_comp_check

,topic_id,nb_comp_0
topic_id,,
0,3882,2261
1,2972,1537
2,2739,132
3,2512,807
4,2155,5
...,...,...
1768,19,0
1769,19,2
1770,18,16


In [22]:
topic_comp_check[topic_comp_check['topic_id'] == topic_comp_check['nb_comp_0']]

,topic_id,nb_comp_0
topic_id,,
984,108,108
1417,57,57
1757,23,23


In [36]:
df_combined[df_combined['topic_id'] == 1699]

,author,date,domain,title,url,content,topic_area,content_processed,distilbert,finbert,roberta,sentiment_match,topic_id,topic_label_str,topic_score,filtered_names_match,nb_companies,nb_comp_0
38270,Sangmi Cha,2020-03-25,reuters,Suspect in South Korea sex blackmail case iden...,https://www.reuters.com/article/us-southkorea-...,South Korean police took the unusual step of p...,business,south korean police took unusual step publicly...,NEGATIVE,NEGATIVE,NEGATIVE,True,1699,,0.488766,[],0,1
117567,https://www.theguardian.com/profile/justinmccurry,2020-05-18,theguardian,South Korean football team apologises for usin...,https://www.theguardian.com/world/2020/may/18/...,A professional football team in South Korea ha...,general,professional football team south korea apologi...,NEGATIVE,NEGATIVE,POSITIVE,False,1699,,0.483769,[],0,1
128507,NaN,2020-05-25,finance.yahoo,South Korean remake of 'Doctor Foster' linked ...,https://finance.yahoo.com/news/south-korean-re...,Adultery is surging in South Korea with worrie...,business,adultery surging south korea worried commentat...,NEGATIVE,NEGATIVE,NEGATIVE,True,1699,,0.438486,[],0,1
184128,NaN,2020-07-02,marketscreener,Fuji : Japanese firm to pay damages for harass...,https://www.marketscreener.com/quote/stock/FUJ...,A Japanese real estate company has been ordere...,business,japanese real estate company ordered pay damag...,NEGATIVE,NEGATIVE,NEGATIVE,True,1699,,0.404688,[],0,1
193678,Associated Press,2020-07-09,marketwatch,"The mayor of Seoul is found dead, half a day a...",https://www.marketwatch.com/(X(1)A(NCs7wM73zAE...,"SEOUL, South Korea (AP) — The missing mayor of...",business,seoul south korea ap missing mayor south korea...,NEGATIVE,NEGATIVE,NEGATIVE,True,1699,,0.840186,[],0,1
193686,Associated Press,2020-07-09,marketwatch,The mayor of Seoul goes missing after one of h...,https://www.marketwatch.com/story/the-mayor-of...,"SEOUL, South Korea (AP) — Authorities searched...",business,seoul south korea ap authorities searched thur...,NEGATIVE,NEGATIVE,NEGATIVE,True,1699,,0.799695,[Park Group plc],1,0
193821,NaN,2020-07-09,finance.yahoo,A corporate building in downtown Seoul lights ...,https://uk.finance.yahoo.com/video/corporate-b...,A corporate building in downtown Seoul lights ...,business,corporate building downtown seoul lights led p...,POSITIVE,NEUTRAL,POSITIVE,False,1699,,0.541716,[],0,1
193836,NaN,2020-07-09,finance.yahoo,Seoul's mayor found dead of unknown causes,https://uk.finance.yahoo.com/news/body-seoul-m...,Seoul City Mayor Park Won-soon was found dead ...,business,seoul city mayor park wonsoon found dead frida...,NEGATIVE,NEGATIVE,NEGATIVE,True,1699,,0.854137,[Park Group plc],1,0
194272,NaN,2020-07-09,finance.yahoo,Seoul mayor reported missing with searches und...,https://uk.finance.yahoo.com/news/seoul-mayor-...,The mayor of South Korean capital Seoul has be...,business,mayor south korean capital seoul reported miss...,NEGATIVE,NEGATIVE,NEGATIVE,True,1699,,0.776621,[],0,1
194652,Amalie Henden,2020-07-09,express,Park Won-soon dead: How did Park Won-soon die?...,https://www.express.co.uk/news/world/1307429/P...,We will use your email address only for sendi...,general,use email address sending newsletters privacy ...,NEGATIVE,NEUTRAL,NEGATIVE,False,1699,,0.772810,[Park Group plc],1,0


In [39]:
df_combined.loc[193686]['content']

'SEOUL, South Korea (AP) — Authorities searched on Thursday for the mayor of South Korea’s capital, Seoul, who vanished amid sexual harassment allegations after giving his daughter a message that she described as sounding like a will, police said. Police said they were looking for Mayor Park Won-soon near a small hill in Seoul’s Sungbuk neighborhood where his cellphone signal was last detected. They said the phone was currently turned off. His daughter called police on Thursday afternoon and said her father had given her “a will-like” verbal message before leaving their home about 4-5 hours earlier. The daughter didn’t explain the contents of the message, said an officer at the Seoul Metropolitan Police Agency who was responsible for the search operation. His daughter said she decided to call police because she couldn’t reach her father on the phone, the officer said, requesting anonymity because she was not authorized to speak to the media about the matter. About 700 officers, three d